<a href="https://colab.research.google.com/github/alessandronicolini/IncrementalLearning/blob/main/LearningWithoutForgetting2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# upload work files from your git hub repository
import sys

!git clone https://github.com/alessandronicolini/IncrementalLearning.git # clone proj repository
!rm -rf IncrementalLearning/README.md 
!rm -rf IncrementalLearning/baselines.ipynb

path = 'IncrementalLearning/'
if path not in sys.path:
    sys.path.append('IncrementalLearning/')

!pip3 install import_ipynb

Cloning into 'IncrementalLearning'...
remote: Enumerating objects: 249, done.
remote: Counting objects: 100% (249/249), done.
remote: Compressing objects: 100% (171/171), done.
remote: Total 249 (delta 126), reused 149 (delta 54), pack-reused 0
Receiving objects: 100% (249/249), 101.50 KiB | 4.41 MiB/s, done.
Resolving deltas: 100% (126/126), done.
  Created wheel for import-ipynb: filename=import_ipynb-0.1.3-cp36-none-any.whl size=2976 sha256=ea8df8227a8d5897d10bebf20d7783635a3a736b256fcf638c7deffa99e92007
  Stored in directory: /root/.cache/pip/wheels/b4/7b/e9/a3a6e496115dffdb4e3085d0ae39ffe8a814eacc44bbf494b5
Successfully built import-ipynb


In [ ]:
import torch
import os
import torch.nn as nn
from torch.utils.data import Subset, DataLoader
import torch.optim as optim
from torchvision import transforms
import numpy as np
import import_ipynb
import copy
# project classes --------------------------------------------------------------
from IncrementalLearning.cifar100 import ilCIFAR100
from resnet_cifar import resnet32

importing Jupyter notebook from /content/IncrementalLearning/cifar100.ipynb


Failed download. Trying https -> http instead. Downloading http://www.cs.toronto.edu/~kriz/cifar-100-python.tar.gz to ./data/cifar-100-python.tar.gz


Extracting ./data/cifar-100-python.tar.gz to ./data
[60, 34, 84, 67, 85, 44, 18, 48, 1, 47, 61, 35, 82, 58, 76, 29, 71, 0, 79, 93, 56, 90, 20, 43, 26, 7, 73, 25, 9, 65, 95, 51, 11, 2, 74, 28, 96, 27, 99, 64, 70, 42, 62, 8, 98, 77, 39, 88, 10, 94, 3, 52, 68, 32, 5, 72, 38, 75, 69, 30, 40, 41, 24, 55, 91, 45, 12, 16, 22, 53, 63, 57, 31, 33, 21, 83, 49, 81, 59, 78, 97, 19, 46, 17, 36, 87, 6, 13, 14, 89, 80, 23, 86, 37, 54, 92, 50, 66, 15, 4]
0
1
2
3
4
5
6
7
8
9

<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>


In [24]:
class lwf(nn.Module):
  def __init__(self,network, random_seed, batch_size):
    super(lwf, self).__init__()
    
    self.batch_size = batch_size
    self.classes_per_task=70
    self.num_tasks=10
    self.LR=2
    self.MOMENTUM=0.9
    self.WEIGHT_DECAY=1e-5
    self.MILESTONES=[49,63]
    self.GAMMA=0.2
    self.numepochs=20
    #self.randomseed = random_seed
    #self.trainloader=None
    #self.testloader=None
    
    self.original_training_set = ilCIFAR100(self.classes_per_task, random_seed)
    self.original_test_set = ilCIFAR100(self.classes_per_task, random_seed, train=False)
    
    self.model = network.to('cuda')
    self.criterion = nn.BCEWithLogitsLoss()
    self.optimizer = optim.SGD(self.model.parameters(), lr=self.LR, momentum=self.MOMENTUM, weight_decay=self.WEIGHT_DECAY)
    self.scheduler = optim.lr_scheduler.MultiStepLR(self.optimizer, milestones=self.MILESTONES, gamma=self.GAMMA)
    
    self.diz = self.original_training_set.get_dict()
    self.task_counter = -1
    #self.classes_seen = 0

  def forward(self, x):
    self.model = self.model.cuda()
    return self.model.forward(x)

  def update_parameters(self, train_dataloader):

    self.task_counter += 1 # new incoming task is starting
    old_model = copy.deepcopy(self)
    old_model.eval()
    old_model.to('cuda')

    for epoch in range(self.numepochs):
        for inputs,labels in train_dataloader:
          labels = torch.tensor([torch.tensor(self.diz[c.item()]) for c in labels])
          inputs=inputs.to('cuda')
          labels=labels.to('cuda')
          self.optimizer.zero_grad()
          outputs=self.model(inputs)

          new_target=torch.eye(100)[labels] 
          new_target=new_target.to('cuda')

          if self.task_counter == 0:
            loss=self.criterion(outputs[:,:self.classes_per_task],new_target[:,:self.classes_per_task])
          
          else:
            seen_classes = self.task_counter*self.classes_per_task
            current_classes = seen_classes + self.classes_per_task

            old_target=old_model(inputs)
            old_target=torch.sigmoid(old_target).cuda()
            target = torch.cat((old_target[:,:seen_classes], new_target[:, seen_classes:current_classes]), dim=1)
            loss=self.criterion(outputs[:,:current_classes],target)

          loss.backward()
          self.optimizer.step()
        self.scheduler.step()

        
  # deep copy the model
  def training_lwf(self):
    
    train_indices = self.original_training_set.get_batch_indexes()
    test_indices = self.original_test_set.get_batch_indexes()
    current_test_indexes=[]
    
    #diz = self.original_training_set.get_dict()
    acc=[]
    accuracy=0

    for i in range(self.num_tasks):
  
      train_dataset=Subset(self.original_training_set,train_indices[i])
      current_test_indexes+=test_indices[i].tolist()
      test_dataset=Subset(self.original_test_set,current_test_indexes)
      train_dataloader = DataLoader(train_dataset, batch_size=self.batch_size, shuffle=True)
      test_dataloader = DataLoader(test_dataset, batch_size=self.batch_size, shuffle=True)        
      self.train()
      self.update_parameters(train_dataloader)
      self.train(False) # Set Network to evaluation mode
      running_corrects = 0
      y_true = []
      y_pred = []
      for inputs, labels in test_dataloader:
          labels = torch.tensor([torch.tensor(self.diz[c.item()]) for c in labels])
          inputs = inputs.to('cuda')
          labels = labels.to('cuda')
          outputs = self.model(inputs)
          _, preds = torch.max(outputs.data, 1)
          running_corrects += torch.sum(preds == labels.data).data.item()
          labels=labels.detach().cpu().numpy()
          labels=labels.tolist()
          y_true.extend(labels)
          preds=preds.detach().cpu().numpy()
          y_pred_tmp = [p for p in preds]
          y_pred.extend(y_pred_tmp)
      accuracy = running_corrects / float(len(test_dataloader.dataset))
      print('Test Accuracy: %.2f' % (100.0 * accuracy))
      print('-' * 80)
      acc.append(accuracy)
    

In [25]:
lwf(network=resnet32(num_classes=100), batch_size = 128, random_seed = 10).training_lwf()


Files already downloaded and verified
[73, 4, 54, 61, 99, 1, 26, 59, 62, 35, 83, 20, 98, 66, 91, 41, 9, 31, 46, 5, 53, 17, 77, 45, 48, 79, 36, 33, 58, 22, 38, 81, 78, 71, 30, 56, 75, 2, 37, 0, 15, 8, 12, 19, 34, 23, 49, 92, 88, 42, 90, 28, 27, 65, 87, 84, 96, 51, 32, 10, 14, 93, 52, 85, 95, 82, 64, 76, 80, 44, 57, 18, 11, 72, 68, 3, 24, 39, 55, 6, 47, 13, 63, 70, 7, 40, 25, 74, 89, 86, 29, 94, 67, 97, 69, 16, 43, 60, 50, 21]
Files already downloaded and verified
[73, 4, 54, 61, 99, 1, 26, 59, 62, 35, 83, 20, 98, 66, 91, 41, 9, 31, 46, 5, 53, 17, 77, 45, 48, 79, 36, 33, 58, 22, 38, 81, 78, 71, 30, 56, 75, 2, 37, 0, 15, 8, 12, 19, 34, 23, 49, 92, 88, 42, 90, 28, 27, 65, 87, 84, 96, 51, 32, 10, 14, 93, 52, 85, 95, 82, 64, 76, 80, 44, 57, 18, 11, 72, 68, 3, 24, 39, 55, 6, 47, 13, 63, 70, 7, 40, 25, 74, 89, 86, 29, 94, 67, 97, 69, 16, 43, 60, 50, 21]
Test Accuracy: 29.46
--------------------------------------------------------------------------------


IndexError: ignored

In [ ]:
model

lwf(
  (model): ResNet(
    (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (layer1): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(16, 16, kernel_size=